This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# "Natural" End Conditions

In the
[notebook about "natural" end conditions for Euclidean splines](../euclidean/end-conditions-natural.ipynb#Bézier-Control-Points)
we have derived the following equations
for calculating the second and penultimate control points
of cubic Bézier splines:

\begin{align*}
\boldsymbol{\tilde{x}}_0^{(+)} &=
\frac{\boldsymbol{x}_0 + \boldsymbol{\tilde{x}}_1^{(-)}}{2} \\
\boldsymbol{\tilde{x}}_{N-1}^{(-)} &=
\frac{\boldsymbol{x}_{N-1} + \boldsymbol{\tilde{x}}_{N-2}^{(+)}}{2}
\end{align*}

These equations can be "translated" to quaternions like this:

\begin{align*}
\tilde{q}_0^{(+)} &=
\left(\tilde{q}_1^{(-)} {q_0}^{-1}\right)^\frac{1}{2} q_0 \\
\tilde{q}_{N-1}^{(-)} &=
\left(\tilde{q}_{N-2}^{(+)} {q_{N-1}}^{-1}\right)^\frac{1}{2} q_{N-1}
\end{align*}

When considering that the control polygon starts with the quaternions
$\left(
q_0, \tilde{q}_0^{(+)}, \tilde{q}_1^{(-)}, q_1, \tilde{q}_1^{(+)}, \dots
\right)$
and ends with
$\left(
\dots, q_{N-2}, \tilde{q}_{N-2}^{(+)}, \tilde{q}_{N-1}^{(-)}, q_{N-1}
\right)$,
we can see that the equations are symmetrical.
The resulting control quaternion is calculated as
the rotation half-way between the first and third control quaternion,
counting either from the beginning ($q_0$) or the end ($q_{N-1}$) of the spline.

In [ ]:
def natural_end_condition(first, third):
    """Return second control quaternion given the first and third.
    
    This also works when counting from the end of the spline.
    
    """
    return first.rotation_to(third)**(1 / 2) * first

## Examples

Let's first import NumPy, a few helpers from [helper.py](helper.py)
and the class
[splines.quaternion.DeCasteljau](../python-module/splines.quaternion.rst#splines.quaternion.DeCasteljau):

In [ ]:
import numpy as np
from helper import angles2quat, animate_rotations, display_animation
from splines.quaternion import DeCasteljau

Furthermore,
let's define a helper function for evaluating a single spline segment:

In [ ]:
def calculate_rotations(control_quaternions):
    times = np.linspace(0, 1, 50)
    return DeCasteljau(
        segments=[control_quaternions],
    ).evaluate(times)

In [ ]:
q0 = angles2quat(45, 0, 0)
q1 = angles2quat(-45, 0, 0)

In [ ]:
q1_control = angles2quat(-45, 0, -90)

In [ ]:
ani = animate_rotations({
    'natural begin': calculate_rotations(
        [q0, natural_end_condition(q0, q1_control), q1_control, q1]),
})

In [ ]:
display_animation(ani, default_mode='reflect')

In [ ]:
q0_control = angles2quat(45, 0, 90)

In [ ]:
ani = animate_rotations({
    'natural end': calculate_rotations(
        [q0, q0_control, natural_end_condition(q1, q0_control), q1]),
})

In [ ]:
display_animation(ani, default_mode='reflect')